# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries
import os
import re
import pandas as pd
import pickle
from sqlalchemy import create_engine

from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.multiclass import OneVsRestClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.metrics import classification_report, f1_score, accuracy_score

In [2]:
# import & download nltk-library
import nltk
from nltk.tokenize import word_tokenize 
from nltk.stem import WordNetLemmatizer
nltk.download(['punkt', 'wordnet', 'stopwords'])

[nltk_data] Downloading package punkt to /home/sfrozy/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /home/sfrozy/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /home/sfrozy/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [3]:
# installing execution time measurement tools 
!pip install ipython-autotime
%load_ext autotime

time: 354 µs (started: 2021-01-05 03:17:50 +01:00)


In [4]:
# load data from database
engine = create_engine('sqlite:///db/disaster_messages_db.db')

sqlquery = "SELECT * FROM disaster_messages;"
df = pd.read_sql(sqlquery, con=engine)
df.head()

,id,message,original,genre,category:related,category:request,category:offer,category:aid_related,category:medical_help,category:medical_products,...,category:aid_centers,category:other_infrastructure,category:weather_related,category:floods,category:storm,category:fire,category:earthquake,category:cold,category:other_weather,category:direct_report
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,1,0,0,1,0,0,...,0,0,1,0,1,0,0,0,0,0
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct,1,1,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


time: 281 ms (started: 2021-01-05 03:17:50 +01:00)


In [5]:
# splitting dataset into X (feature-data) and Y (label-data)
X = df['message']
Y = df.iloc[:, 4:] # the last 36(categories)-column
print(f'X-shape {X.shape} and Y-shape {Y.shape}')

X-shape (26028,) and Y-shape (26028, 35)
time: 1.58 ms (started: 2021-01-05 03:17:50 +01:00)


### 2. Write a tokenization function to process your text data

In [6]:
url_regex = 'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'

def tokenize(text: str, url_regex:str = url_regex):
    # get list of all urls using regex
    detected_urls = re.findall(url_regex, text)
    
    # replace each url in text string with placeholder
    for url in detected_urls:
        text = text.replace(url, 'placeholder')

    # tokenize text
    tokens = word_tokenize(text)
    
    # initiate lemmatizer
    lemmatizer = WordNetLemmatizer()

    # iterate through each token
    clean_tokens = []
    for tok in tokens:
        
        # lemmatize, normalize case, and remove leading/trailing white space
        clean_tok = lemmatizer.lemmatize(tok).lower().strip()
        clean_tokens.append(clean_tok)

    return clean_tokens

time: 11.4 ms (started: 2021-01-05 03:17:50 +01:00)


In [7]:
X[1]

'Is the Hurricane over or is it not over'

time: 8.09 ms (started: 2021-01-05 03:17:50 +01:00)


In [8]:
tokenize(X[1])

['is', 'the', 'hurricane', 'over', 'or', 'is', 'it', 'not', 'over']

time: 974 ms (started: 2021-01-05 03:17:50 +01:00)


### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [9]:
rfc_pipeline = Pipeline([
        ('text_pipeline', Pipeline([
            ('vect', CountVectorizer(tokenizer=tokenize)),
            ('tfidf', TfidfTransformer())
        ])),
        ('rfc_classifier', MultiOutputClassifier(RandomForestClassifier(n_jobs=-1)))
    ])

time: 404 µs (started: 2021-01-05 03:17:51 +01:00)


### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [14]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, random_state=16)

time: 7.71 ms (started: 2021-01-05 03:18:38 +01:00)


In [15]:
rfc_pipeline.fit(X_train, Y_train)

Pipeline(steps=[('text_pipeline',
                 Pipeline(steps=[('vect',
                                  CountVectorizer(tokenizer=<function tokenize at 0x7fcb1b7d7040>)),
                                 ('tfidf', TfidfTransformer())])),
                ('rfc_classifier',
                 MultiOutputClassifier(estimator=RandomForestClassifier(n_jobs=-1)))])

time: 27.7 s (started: 2021-01-05 03:18:40 +01:00)


### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [ ]:
# predict with the pipeline

In [16]:
def evaluate_model(model, X_test, Y_test, category_names):
    Y_pred = model.predict(X_test)

    # iterating for each target columns to calculate the f1-score, precision, and accuracy.
    for i in range(len(category_names)):
        print(f"----Classification Report for category :{Y.columns[i]}\n")
        print(classification_report(Y_test.iloc[:, i].values, Y_pred[:, i]))
        print(f'Accuracy {accuracy_score(Y_test.iloc[:, i].values, Y_pred[:, i])}\n')
        print(f"F1 {f1_score(Y_test.iloc[:, i].values, Y_pred[:, i],average='weighted')}\n")

evaluate_model(rfc_pipeline, X_test, Y_test, list(Y.columns))

----Classification Report for category :category:related

              precision    recall  f1-score   support

           0       0.73      0.26      0.38      1551
           1       0.81      0.97      0.88      4956

    accuracy                           0.80      6507
   macro avg       0.77      0.61      0.63      6507
weighted avg       0.79      0.80      0.76      6507

Accuracy 0.8006761948670662

F1 0.7620873232219401

----Classification Report for category :category:request

              precision    recall  f1-score   support

           0       0.90      0.99      0.94      5411
           1       0.89      0.45      0.60      1096

    accuracy                           0.90      6507
   macro avg       0.90      0.72      0.77      6507
weighted avg       0.90      0.90      0.88      6507

Accuracy 0.8978023666820347

F1 0.8831950472560569

----Classification Report for category :category:offer

              precision    recall  f1-score   support

           0   

/home/sfrozy/anaconda3/envs/disaster_response_pipeline/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [17]:
def display_accuracy(model, Y_test):
    Y_pred = model.predict(X_test)

    accuracy = (Y_pred == Y_test.values).mean()
    print('The model accuracy is {:.3f}'.format(accuracy))

display_accuracy(rfc_pipeline, Y_test)

The model accuracy is 0.944
time: 2.96 s (started: 2021-01-05 03:19:18 +01:00)


### 6. Improve your model
Use grid search to find better parameters. 

In [ ]:
list(rfc_pipeline.get_params().keys())

In [ ]:
parameters = {
        'text_pipeline__tfidf__use_idf': (True, False),
        'rfc_classifier__estimator__n_estimators': [10, 20]
}
cv = GridSearchCV(rfc_pipeline, n_jobs=-1, param_grid=parameters)
cv

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [ ]:
cv.fit(X_train, Y_train)

In [ ]:
Y_pred = cv.predict(X_test)

In [ ]:
evaluate_model(cv, X_test, Y_test, list(Y.columns))

In [ ]:
display_accuracy(cv, Y_test)

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

=> some Classifier Idea:
- AdaBoostClassifier : AdaBoostClassifier()
- KNN Classifier : KNeighborsClassifier()
- Decision Tree Classifier : DecisionTreeClassifier()
- SVM Classifier : SVC()
- Naive Bayes Classifier : MultinomialNB()

=> but we will try only AdaBoosClassifier, and choose between AdaBoostClassifier or RandomForest

---

In [19]:
ada_pipeline = Pipeline([
        ('text_pipeline', Pipeline([
            ('vect', CountVectorizer(tokenizer=tokenize)),
            ('tfidf', TfidfTransformer())
        ])),
        ('classifier', MultiOutputClassifier(AdaBoostClassifier()))
    ])

ada_pipeline.fit(X_train, Y_train)
#evaluate_model(ada_pipeline, X_test, Y_test, list(Y.columns))
display_accuracy(ada_pipeline, Y_test)

The model accuracy is 0.947
time: 1min 24s (started: 2021-01-05 03:21:18 +01:00)


=> The accuracy of ADA Classifier is **higher** than the RandomForestClassifier. (0.944)

---

In [20]:
knn_pipeline = Pipeline([
        ('text_pipeline', Pipeline([
            ('vect', CountVectorizer(tokenizer=tokenize)),
            ('tfidf', TfidfTransformer())
        ])),
        ('classifier', MultiOutputClassifier(KNeighborsClassifier(n_jobs=-1)))
    ])

knn_pipeline.fit(X_train, Y_train)
#evaluate_model(knn_pipeline, X_test, Y_test, list(Y.columns))
display_accuracy(knn_pipeline, Y_test)

The model accuracy is 0.924
time: 1min 36s (started: 2021-01-05 02:55:37 +01:00)


=> The accuracy of KNN Classifier is **lower** than the RandomForestClassifier. (0.944)

---

In [22]:
dt_pipeline = Pipeline([
        ('text_pipeline', Pipeline([
            ('vect', CountVectorizer(tokenizer=tokenize)),
            ('tfidf', TfidfTransformer())
        ])),
        ('classifier', MultiOutputClassifier(DecisionTreeClassifier()))
    ])

dt_pipeline.fit(X_train, Y_train)
#evaluate_model(dt_pipeline, X_test, Y_test, list(Y.columns))
display_accuracy(dt_pipeline, Y_test)

The model accuracy is 0.929
time: 4min 28s (started: 2021-01-05 02:58:28 +01:00)


=> The accuracy of Decision Tree Classifier is **lower** than the RandomForestClassifier. (0.944)

---


In [18]:
mb_pipeline = Pipeline([
        ('text_pipeline', Pipeline([
            ('vect', CountVectorizer(tokenizer=tokenize)),
            ('tfidf', TfidfTransformer())
        ])),
        ('classifier', OneVsRestClassifier(MultinomialNB()))
    ])

mb_pipeline.fit(X_train, Y_train)
#evaluate_model(mb_pipeline, X_test, Y_test, list(Y.columns))
display_accuracy(mb_pipeline, Y_test)

The model accuracy is 0.933
time: 5.88 s (started: 2021-01-05 03:19:32 +01:00)


=> The accuracy of MultinomialNB Classifier is **lower** than the RandomForestClassifier. (0.944)

---

**Conclusion**
=> Therefore we will use ADA Classifier to train the model.


In [20]:
list(ada_pipeline.get_params().keys())

['memory',
 'steps',
 'verbose',
 'text_pipeline',
 'classifier',
 'text_pipeline__memory',
 'text_pipeline__steps',
 'text_pipeline__verbose',
 'text_pipeline__vect',
 'text_pipeline__tfidf',
 'text_pipeline__vect__analyzer',
 'text_pipeline__vect__binary',
 'text_pipeline__vect__decode_error',
 'text_pipeline__vect__dtype',
 'text_pipeline__vect__encoding',
 'text_pipeline__vect__input',
 'text_pipeline__vect__lowercase',
 'text_pipeline__vect__max_df',
 'text_pipeline__vect__max_features',
 'text_pipeline__vect__min_df',
 'text_pipeline__vect__ngram_range',
 'text_pipeline__vect__preprocessor',
 'text_pipeline__vect__stop_words',
 'text_pipeline__vect__strip_accents',
 'text_pipeline__vect__token_pattern',
 'text_pipeline__vect__tokenizer',
 'text_pipeline__vect__vocabulary',
 'text_pipeline__tfidf__norm',
 'text_pipeline__tfidf__smooth_idf',
 'text_pipeline__tfidf__sublinear_tf',
 'text_pipeline__tfidf__use_idf',
 'classifier__estimator__algorithm',
 'classifier__estimator__base_es

time: 2.93 ms (started: 2021-01-05 03:23:27 +01:00)


### 9. Export your model as a pickle file

In [61]:
model_base_folder = 'models'
if not os.path.exists(model_base_folder):
    os.makedirs(model_base_folder)

# create a pickle file for the model
file_name = f'{model_base_folder}/classifier.pkl'
with open (file_name, 'wb') as f:
    pickle.dump(cv, f)
    f.close()

time: 783 ms (started: 2021-01-05 01:07:19 +01:00)


### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.

In [ ]:
# see `models/train_classifier.py`

### 11. Prepare query for the flask visualization
- Frequency of the category in the dataset

In [59]:
def calculate_target_frequency(y):
    cats = []
    frequency = []
    for col in Y.columns:
        print(f'{col[9:]} has {Y[col][Y[col] == True].sum()} ')
        cats.append(col[9:])
        frequency.append(Y[col][Y[col] == True].sum())

    return pd.DataFrame({'category': cats, 'frequency': frequency})



related has 19906 
request has 4474 
offer has 118 
aid_related has 10860 
medical_help has 2084 
medical_products has 1313 
search_and_rescue has 724 
security has 471 
military has 860 
child_alone has 0 
water has 1672 
food has 2923 
shelter has 2314 
clothing has 405 
money has 604 
missing_people has 298 
refugees has 875 
death has 1194 
other_aid has 3446 
infrastructure_related has 1705 
transport has 1201 
buildings has 1333 
electricity has 532 
tools has 159 
hospitals has 283 
shops has 120 
aid_centers has 309 
other_infrastructure has 1151 
weather_related has 7297 
floods has 2155 
storm has 2443 
fire has 282 
earthquake has 2455 
cold has 530 
other_weather has 1376 
direct_report has 5075 
time: 27 ms (started: 2021-01-04 23:45:27 +01:00)


,category,frequency
0,related,19906
1,request,4474
2,offer,118
3,aid_related,10860
4,medical_help,2084
